<a href="https://colab.research.google.com/github/resloved/RWKV-notebooks/blob/master/RWKV_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RWKV LangChain

[RWKV](https://github.com/BlinkDL/RWKV-LM) is an RNN with transformer-level performance


This notebook aims to demo the use of RWKV in conjunction with [LangChain](https://github.com/hwchase17/langchain)

Extending the work by [jiamingkong](https://github.com/jiamingkong) on [github](https://gist.github.com/jiamingkong/3c2b4529e602c0178a7c6fcd10b7c639)

In [ ]:
#@title Google Drive Options { display-mode: "form" }
save_models_to_drive = True #@param {type:"boolean"}
drive_mount = '/content/drive' #@param {type:"string"}
model_dir = 'rwkv-models' #@param {type:"string"}

import os
if save_models_to_drive:
    from google.colab import drive
    drive.mount(drive_mount)
    model_dir_path = f"{drive_mount}/MyDrive/{model_dir}" if save_models_to_drive else f"/content/{model_dir}"
else:
    model_dir_path = "/content"

os.makedirs(f"{model_dir_path}", exist_ok=True)

print(f"Saving models to {model_dir_path}")

In [ ]:
!pip install langchain rwkv ninja

In [ ]:
!git clone https://github.com/BlinkDL/RWKV-LM

In [ ]:
#@title Select/Download Model { display-mode: "form" }
import urllib

#@markdown Select the model you'd like to use:
model_file = "RWKV-4-Raven-3B-v11-Eng99%-Other1%-20230425-ctx4096.pth" #@param {type:"string"}
#@markdown It will first search `model_dir` for `model_file`.
#@markdown If it isn't valid path, it will attempt to download a `RWKV-v4-Raven` model from huggingface.
#@markdown To see which options you have, take a look at the [repo](https://huggingface.co/BlinkDL/rwkv-4-raven/).

#@markdown ---

#@markdown For example:
#@markdown - RWKV-v4-Raven-14B-v11x: `RWKV-4-Raven-14B-v11x-Eng99%-Other1%-20230501-ctx8192.pth`
#@markdown - RWKV-v4-Raven-7B-v11x: `RWKV-4-Raven-7B-v11x-Eng99%-Other1%-20230429-ctx8192.pth`
#@markdown - RWKV-v4-Raven-3B-v11: `RWKV-4-Raven-3B-v11-Eng99%-Other1%-20230425-ctx4096.pth`
#@markdown - RWKV-v4-Raven-1B5-v11: `RWKV-4-Raven-1B5-v11-Eng99%-Other1%-20230425-ctx4096.pth`
#@markdown - Custom Model: `/rwkv-subdirectory/custom-rwkv.pth`

model_path = f"{model_dir_path}/{model_file}"
if not os.path.exists(model_path):
    model_repo = f"https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main"
    model_url = f"{model_repo}/{urllib.parse.quote_plus(model_file)}"
    try:
        print(f"Downloading '{model_file}' from {model_url} this may take a while")
        urllib.request.urlretrieve(model_url, model_path)
        print(f"Using {model_path} as base")
    except Exception as e:
        print(f"Model '{model_file}' doesn't exist")
        raise Exception
else:
    print(f"Using {model_path} as base")

In [ ]:
#@title Load Model
import os
os.environ["RWKV_CUDA_ON"] = '1'
os.environ["RWKV_JIT_ON"] = '1'

from langchain.llms import RWKV

strategy = "cuda fp16i8 *20 -> cuda fp16" #@param {"type":"string"}
model = RWKV(model=model_path, strategy=strategy, tokens_path="RWKV-LM/RWKV-v4/20B_tokenizer.json")

In [ ]:
#@title Chain
#@markdown A simple chain example. You first create the instruction template, and feed in your prompt as the instruction variable.

from langchain.prompts import PromptTemplate
task = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
# Instruction:
{instruction}

# Response:
""" 
instruction = "What is a good company name for product of superglue?" #@param {type:"string"}

prompt = PromptTemplate(
    input_variables=["instruction"],
    template=task,
)

from langchain.chains import LLMChain
chain = LLMChain(llm=model, prompt=prompt)

print(chain.run(instruction))

#@markdown Documentation — 
#@markdown [PromptTemplate](https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/prompt_serialization.html), 
#@markdown [LLMChain](https://python.langchain.com/en/latest/modules/chains/generic/llm_chain.html)

In [ ]:
#@title Conversation Chain
#@markdown Using a ConversationChain you are able to continue the chain based on a simple conversation structure.

reset_conversation = False #@param {type:"boolean"}
prompt = "Why not?" #@param {type:"string"}
from langchain import ConversationChain

if not 'conversation' in locals() or reset_conversation:
    conversation = ConversationChain(llm=model, verbose=True)

print(conversation.predict(input=prompt, stop="\n").strip())

#@markdown Documentation —
#@markdown [ConversationChain](https://python.langchain.com/en/latest/modules/memory/getting_started.html#using-in-a-chain)

In [ ]:
#@title Sequential Chain
#@markdown In this example we use the SequentialChain object to first create a task to create a synopsis for a play,
#@markdown then sequentially chain that result into another task to review the play.
#@markdown 
from langchain.prompts import PromptTemplate
task = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
# Instruction:
{instruction}

# Response:
""" 

model.max_tokens_per_generation = 1000
template = """
You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.
Title: {title}
Era: {era}
"""

response = "Playwright: This is a synopsis for the above play:"

template = task.format(instruction=template, response=response)
prompt_template = PromptTemplate(template=template, input_variables=["title", 'era'])
synopsis_chain = LLMChain(llm=model, prompt=prompt_template, output_key="synopsis")

template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis:
{synopsis}
"""
response = "Review from a New York Times play critic of the above play:"
template = task.format(instruction = template, response=response)
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=model, prompt=prompt_template, output_key="review")

from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

title = "Tragedy at sunset on the beach" #@param {type:"string"}
era = "Victorian England" #@param {type:"string"}

bold_start = "\033[1m"
bold_stop = "\033[0m"
result = overall_chain({"title": title, "era": era})

print(f"""
{bold_start}> Prompt{bold_stop}

{title} – {era}

{bold_start}> Synopsis{bold_stop}

{result['synopsis']}

{bold_start}> Review{bold_stop}

{result['review']}
""")

#@markdown Documentation — [SequentialChain](https://python.langchain.com/en/latest/modules/chains/generic/sequential_chains.html)